In [157]:
def read_file(file_name):
	with open(file_name, "r") as f:
		data = f.read().splitlines()
	return data

In [158]:
EXAMPLE_FILE = "example"
INPUT_FILE = "input"
time_left = 30

In [159]:
from dataclasses import dataclass

@dataclass
class Valve:
	name: str
	flow_rate: int
	tunnels: list

In [160]:
def prepare_data(file_name):
	valves = {}
	data = read_file(file_name)
	for line in data:
		line = line.split(' ')
		valves[line[1]] = Valve(line[1], int(line[4][5:-1]), [v[:-1] if len(v) > 2 else v for v in line[9:]])
	return valves

example_valves = prepare_data(EXAMPLE_FILE)

In [161]:
def valve_distance(valves, start: str, end: str):
	print(f"distance {start} -> {end}")
	if start == end:
		return 0
	if end in valves[start].tunnels:
		return 1
	distance = 0
	queue = [start]
	new_queue = []
	while end not in queue:
		new_queue.extend(valves[queue.pop(0)].tunnels)
		if not queue:
			queue = new_queue
			new_queue = []
			distance += 1
	return distance

In [162]:
def relative_score(valves, pos: str):
	scores = {}
	for valve in valves.values():
		print(valve)
		score = valve.flow_rate - valve_distance(valves, pos, valve.name)
		print(score)
		scores[valve.name] = score
	return scores

relative_score(example_valves, 'AA')

Valve(name='AA', flow_rate=0, tunnels=['DD', 'II', 'BB'])
distance AA -> AA
0
Valve(name='BB', flow_rate=13, tunnels=['CC', 'AA'])
distance AA -> BB
12
Valve(name='CC', flow_rate=2, tunnels=['DD', 'BB'])
distance AA -> CC
0
Valve(name='DD', flow_rate=20, tunnels=['CC', 'AA', 'EE'])
distance AA -> DD
19
Valve(name='EE', flow_rate=3, tunnels=['FF', 'DD'])
distance AA -> EE
1
Valve(name='FF', flow_rate=0, tunnels=['EE', 'GG'])
distance AA -> FF
-3
Valve(name='GG', flow_rate=0, tunnels=['FF', 'HH'])
distance AA -> GG
-4
Valve(name='HH', flow_rate=22, tunnels=['GG'])
distance AA -> HH
17
Valve(name='II', flow_rate=0, tunnels=['AA', 'JJ'])
distance AA -> II
-1
Valve(name='JJ', flow_rate=21, tunnels=['II'])
distance AA -> JJ
19


{'AA': 0,
 'BB': 12,
 'CC': 0,
 'DD': 19,
 'EE': 1,
 'FF': -3,
 'GG': -4,
 'HH': 17,
 'II': -1,
 'JJ': 19}

try to find strongly connented parts

In [163]:
def visit(valve, visited, valves, queue):
	if valve.name in visited:
		return 
	print(f"visit {valve.name}", visited)
	visited.add(valve.name)
	for nbr in valve.tunnels:
		if nbr not in visited:
			visit(valves[nbr], visited, valves, queue)
		queue.insert(0, valve.name)


In [164]:
def assign(valve, root, valves, assignments):
	if valve in assignments.keys():
		return
	print(f"assign {valve} to {root}")
	assignments[valve] = root
	for nbr in valves[valve].tunnels:
		assign(nbr, root, valves, assignments)

In [167]:
def strongly_connected(valves):
	visited = set()
	queue = []
	assignments = {}
	for valve in valves.values():
		visit(valve, visited, valves, queue)
	print(queue)
	for valve in queue:
		assign(valve, valve, valves, assignments)
	print(assignments.values())

strongly_connected(example_valves)

visit AA set()
visit DD {'AA'}
visit CC {'DD', 'AA'}
visit BB {'CC', 'DD', 'AA'}
visit EE {'CC', 'DD', 'AA', 'BB'}
visit FF {'CC', 'AA', 'EE', 'DD', 'BB'}
visit GG {'CC', 'AA', 'EE', 'DD', 'BB', 'FF'}
visit HH {'CC', 'AA', 'EE', 'DD', 'GG', 'BB', 'FF'}
visit II {'CC', 'AA', 'EE', 'DD', 'GG', 'BB', 'HH', 'FF'}
visit JJ {'CC', 'AA', 'EE', 'DD', 'GG', 'BB', 'HH', 'II', 'FF'}
['AA', 'AA', 'II', 'JJ', 'II', 'AA', 'DD', 'EE', 'EE', 'FF', 'GG', 'HH', 'GG', 'FF', 'DD', 'DD', 'CC', 'BB', 'BB', 'CC']
assign AA to AA
assign DD to AA
assign CC to AA
assign BB to AA
assign EE to AA
assign FF to AA
assign GG to AA
assign HH to AA
assign II to AA
assign JJ to AA
dict_values(['AA', 'AA', 'AA', 'AA', 'AA', 'AA', 'AA', 'AA', 'AA', 'AA'])


strongly connented parts does not seem helpful :C

In [166]:
def solve_part1(file_name):
	valves = prepare_data(file_name)
	valve = 'AA'
	for v in valves:
		print(v, valve_distance(valves, valve, v))

solve_part1(EXAMPLE_FILE)

distance AA -> AA
AA 0
distance AA -> BB
BB 1
distance AA -> CC
CC 2
distance AA -> DD
DD 1
distance AA -> EE
EE 2
distance AA -> FF
FF 3
distance AA -> GG
GG 4
distance AA -> HH
HH 5
distance AA -> II
II 1
distance AA -> JJ
JJ 2
